In [1]:
import pandas as pd
import numpy as np

import time
from datetime import datetime
from datetime import timedelta

import os

In [2]:
current_folder = os.path.join(os.getcwd(),'data')

In [3]:
files_list = os.listdir(current_folder)

In [4]:
data = pd.DataFrame()
for file in files_list:
    current_file = pd.read_csv(os.path.join(current_folder,file), encoding="latin-1")
    file_name = file[:file.find('_(')]

    current_file.drop(['Waluta'], axis = 1, inplace = True)
    try:
        data = pd.merge(data, current_file, left_on = 'Data',  right_on = 'Data', suffixes=('', file_name))
    except:
        data = current_file
    

In [5]:
columns = []
for column in data.columns:
    columns.append(column.replace('Wartoæ dla ', ''))

In [6]:
data.columns=columns

In [7]:
backup_data = data

In [8]:
data = backup_data

In [9]:
data

,Data,Goldman Sachs Japonia (K),Goldman Sachs Zrównowa¿ony (A),Goldman Sachs Stabilny Globalnej Dywersyfikacji (A),Goldman Sachs Konserwatywny (A)
0,2014-03-24,140.33,278.99,100.00,250.78
1,2014-03-25,140.38,280.70,100.28,250.79
2,2014-03-26,141.59,282.48,100.25,250.80
3,2014-03-27,142.79,282.57,100.49,250.77
4,2014-03-28,143.98,283.37,100.63,250.82
...,...,...,...,...,...
2529,2024-05-06,386.37,450.90,137.31,310.13
2530,2024-05-07,390.50,450.48,137.71,310.05
2531,2024-05-08,386.59,451.51,137.56,310.07
2532,2024-05-09,388.34,452.61,137.68,310.13


In [10]:
origin_columns = data.columns

In [11]:
on = origin_columns[1:]

In [12]:
for column in on:
    name = column[:column.find(" (")]
    # Obliczamy cumulative sum
    data[name+'Suma'] = data[column].cumsum()

    # Obliczamy kroczące średnie dla 2, 5, 10 i 30 dni.
    data[name+'Krocząca Średnia [2d]']   = data[column].rolling(window=2).mean()
    data[name+'Krocząca Średnia [5d]']   = data[column].rolling(window=5).mean()
    data[name+'Krocząca Średnia [10d]']  = data[column].rolling(window=10).mean()
    data[name+'Krocząca Średnia [30d]']  = data[column].rolling(window=30).mean()
    data[name+'Krocząca Średnia [60d]']  = data[column].rolling(window=60).mean()
    data[name+'Krocząca Średnia [120d]']  = data[column].rolling(window=120).mean()
    data[name+'Krocząca Średnia [360d]']  = data[column].rolling(window=360).mean()

    data[name+'Pierwsza pochodna'] = data[column].diff()
    data[name+'Druga pochodna'] = data[column].diff().diff()

In [13]:
data

,Data,Goldman Sachs Japonia (K),Goldman Sachs Zrównowa¿ony (A),Goldman Sachs Stabilny Globalnej Dywersyfikacji (A),Goldman Sachs Konserwatywny (A),Goldman Sachs JaponiaSuma,Goldman Sachs JaponiaKrocząca Średnia [2d],Goldman Sachs JaponiaKrocząca Średnia [5d],Goldman Sachs JaponiaKrocząca Średnia [10d],Goldman Sachs JaponiaKrocząca Średnia [30d],...,Goldman Sachs KonserwatywnySuma,Goldman Sachs KonserwatywnyKrocząca Średnia [2d],Goldman Sachs KonserwatywnyKrocząca Średnia [5d],Goldman Sachs KonserwatywnyKrocząca Średnia [10d],Goldman Sachs KonserwatywnyKrocząca Średnia [30d],Goldman Sachs KonserwatywnyKrocząca Średnia [60d],Goldman Sachs KonserwatywnyKrocząca Średnia [120d],Goldman Sachs KonserwatywnyKrocząca Średnia [360d],Goldman Sachs KonserwatywnyPierwsza pochodna,Goldman Sachs KonserwatywnyDruga pochodna
0,2014-03-24,140.33,278.99,100.00,250.78,140.33,NaN,NaN,NaN,NaN,...,250.78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014-03-25,140.38,280.70,100.28,250.79,280.71,140.355,NaN,NaN,NaN,...,501.57,250.785,NaN,NaN,NaN,NaN,NaN,NaN,0.01,NaN
2,2014-03-26,141.59,282.48,100.25,250.80,422.30,140.985,NaN,NaN,NaN,...,752.37,250.795,NaN,NaN,NaN,NaN,NaN,NaN,0.01,2.842171e-14
3,2014-03-27,142.79,282.57,100.49,250.77,565.09,142.190,NaN,NaN,NaN,...,1003.14,250.785,NaN,NaN,NaN,NaN,NaN,NaN,-0.03,-4.000000e-02
4,2014-03-28,143.98,283.37,100.63,250.82,709.07,143.385,141.814,NaN,NaN,...,1253.96,250.795,250.792,NaN,NaN,NaN,NaN,NaN,0.05,8.000000e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2529,2024-05-06,386.37,450.90,137.31,310.13,543040.37,386.400,385.602,382.289,388.460333,...,682590.79,310.050,309.912,309.670,309.053667,308.064667,305.519417,294.959083,0.16,7.000000e-02
2530,2024-05-07,390.50,450.48,137.71,310.05,543430.87,388.435,387.180,383.639,388.451667,...,682900.84,310.090,309.974,309.755,309.120667,308.132167,305.602167,295.057444,-0.08,-2.400000e-01
2531,2024-05-08,386.59,451.51,137.56,310.07,543817.46,388.545,387.986,384.445,387.984667,...,683210.91,310.060,310.020,309.828,309.185333,308.198833,305.684333,295.155917,0.02,1.000000e-01
2532,2024-05-09,388.34,452.61,137.68,310.13,544205.80,387.465,387.646,385.262,387.610667,...,683521.04,310.100,310.070,309.898,309.251000,308.265000,305.767500,295.254417,0.06,4.000000e-02


In [85]:
data.to_csv(os.path.join(current_folder,'data.csv'))

In [4]:
import pathlib
import pandas as pd

In [14]:


df = pd.read_csv("/Users/mdebosz/Documents/Prywatne/Python-Scripts/GSTFI/GSTFI/src/data/data.csv")

columns = list(df.columns)
columns= [{'label': name, 'value':name} for name in columns][1:]

In [15]:
columns

[{'label': 'Data', 'value': 'Data'},
 {'label': 'Goldman Sachs Japonia (K)', 'value': 'Goldman Sachs Japonia (K)'},
 {'label': 'Goldman Sachs Zrównowa¿ony (A)',
  'value': 'Goldman Sachs Zrównowa¿ony (A)'},
 {'label': 'Goldman Sachs Stabilny Globalnej Dywersyfikacji (A)',
  'value': 'Goldman Sachs Stabilny Globalnej Dywersyfikacji (A)'},
 {'label': 'Goldman Sachs Konserwatywny (A)',
  'value': 'Goldman Sachs Konserwatywny (A)'},
 {'label': 'Goldman Sachs JaponiaSuma', 'value': 'Goldman Sachs JaponiaSuma'},
 {'label': 'Goldman Sachs JaponiaKrocząca Średnia [2d]',
  'value': 'Goldman Sachs JaponiaKrocząca Średnia [2d]'},
 {'label': 'Goldman Sachs JaponiaKrocząca Średnia [5d]',
  'value': 'Goldman Sachs JaponiaKrocząca Średnia [5d]'},
 {'label': 'Goldman Sachs JaponiaKrocząca Średnia [10d]',
  'value': 'Goldman Sachs JaponiaKrocząca Średnia [10d]'},
 {'label': 'Goldman Sachs JaponiaKrocząca Średnia [30d]',
  'value': 'Goldman Sachs JaponiaKrocząca Średnia [30d]'},
 {'label': 'Goldman Sach

In [9]:
[print(name) for name in columns]

label
value


[None, None]

In [16]:
columns = list(df.columns)

In [17]:
columns

['Unnamed: 0',
 'Data',
 'Goldman Sachs Japonia (K)',
 'Goldman Sachs Zrównowa¿ony (A)',
 'Goldman Sachs Stabilny Globalnej Dywersyfikacji (A)',
 'Goldman Sachs Konserwatywny (A)',
 'Goldman Sachs JaponiaSuma',
 'Goldman Sachs JaponiaKrocząca Średnia [2d]',
 'Goldman Sachs JaponiaKrocząca Średnia [5d]',
 'Goldman Sachs JaponiaKrocząca Średnia [10d]',
 'Goldman Sachs JaponiaKrocząca Średnia [30d]',
 'Goldman Sachs JaponiaKrocząca Średnia [60d]',
 'Goldman Sachs JaponiaKrocząca Średnia [120d]',
 'Goldman Sachs JaponiaKrocząca Średnia [360d]',
 'Goldman Sachs JaponiaPierwsza pochodna',
 'Goldman Sachs JaponiaDruga pochodna',
 'Goldman Sachs Zrównowa¿onySuma',
 'Goldman Sachs Zrównowa¿onyKrocząca Średnia [2d]',
 'Goldman Sachs Zrównowa¿onyKrocząca Średnia [5d]',
 'Goldman Sachs Zrównowa¿onyKrocząca Średnia [10d]',
 'Goldman Sachs Zrównowa¿onyKrocząca Średnia [30d]',
 'Goldman Sachs Zrównowa¿onyKrocząca Średnia [60d]',
 'Goldman Sachs Zrównowa¿onyKrocząca Średnia [120d]',
 'Goldman Sachs Zr

In [22]:
[*columns[0]]

['U', 'n', 'n', 'a', 'm', 'e', 'd', ':', ' ', '0']